In [1]:
from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.connections import Connection, Bus
from tespy.tools import ExergyAnalysis
from CoolProp.CoolProp import PropsSI as cpsi

#fluids
wf = 'R600a'#coolprop
fld_wf = {wf:1} #tespy

#network
refrigerator = Network(T_unit = 'C', p_unit = 'bar', h_unit = 'kJ / kg', m_unit = 'kg / s')

#components
src_wf = Source('working-fluid-source')
snk_wf = Sink('working-fluid-sink')
cmp_th = Valve('throttle')
cmp_ev = SimpleHeatExchanger('evaporator', dissipative = False)
cmp_cp = Compressor('compressor')
cmp_co = SimpleHeatExchanger('condensor')
cmp_cc = CycleCloser('cycle-closer')

#connection

c03   = Connection(cmp_cc, 'out1', cmp_th, 'in1', label = '03')
c04   = Connection(cmp_th, 'out1', cmp_ev, 'in1', label = '04')
c01   = Connection(cmp_ev, 'out1', cmp_cp, 'in1', label = '01')
c02   = Connection(cmp_cp, 'out1', cmp_co, 'in1', label = '02')
c02cc = Connection(cmp_co, 'out1', cmp_cc, 'in1', label = '02cc')

refrigerator.add_conns(c01, c02, c02cc, c03, c04)

#parameters

#components
cmp_ev.set_attr(Q = 150, pr = 1)
cmp_cp.set_attr(eta_s = 0.75)
cmp_co.set_attr(pr = 1)

#connection
#c03.set_attr(T=40, fluid = fld_wf)
h_li = cpsi('H', 'Q', 0, 'T', 313.15, wf)/1e3
c02cc.set_attr(h = h_li, fluid = fld_wf)

p_sat = cpsi('P', 'Q', 1, 'T', 253.15, wf)/1e5
c04.set_attr(p = p_sat)

h_sh = cpsi('H', 'P', p_sat*1e5, 'T', 255.15, wf)/1e3
c01.set_attr(h=h_sh)

c02.set_attr(T=60)

#power and heat busses
motor = Bus('motor')
motor.add_comps({'comp' : cmp_cp, 'char' : 0.95, 'base' : 'bus'})

cooling = Bus('cooling heat')
cooling.add_comps({'comp' : cmp_ev, 'base' : 'bus'})

refrigerator.add_busses(motor, cooling)

#solve
refrigerator.solve(mode='design')
refrigerator.print_results()

#print COP
print('COP:', cooling.P.val/motor.P.val)

#exergy analysis
ean = ExergyAnalysis(network=refrigerator, E_F = [motor], E_P = [cooling])
ean.analyse(1.013, 25)
ean.print_results()


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 6.44e+05   | 2 %        | 1.78e+00   | 3.72e+05   | 2.04e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.58e+04   | 16 %       | 2.82e-02   | 4.37e+05   | 8.20e+03   | 0.00e+00   | 0.00e+00   
 3     | 7.34e+03   | 23 %       | 4.85e-19   | 1.30e+05   | 3.22e+03   | 0.00e+00   | 0.00e+00   
 4     | 3.02e+02   | 39 %       | 0.00e+00   | 5.02e+03   | 1.79e+02   | 0.00e+00   | 0.00e+00   
 5     | 3.83e-01   | 71 %       | 0.00e+00   | 5.53e+00   | 2.66e-01   | 0.00e+00   | 0.00e+00   
 6     | 4.58e-07   | 100 %      | 0.00e+00   | 5.84e-06   | 3.52e-07   | 0.00e+00   | 0.00e+00   
 7     | 3.20e-10   | 100 %      | 0.00e+00   | 6.29e-09   | 1.36e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 7, Calculation time: 0.03 s, Iterations per second: 222.54

##### RESULTS (Compressor) ###

In [3]:
# instead of pastille and controller, around component we need source and sink (before and after component),
# source and sink are proper components